# Test Statistical Analysis Function
This notebook tests `stat_ip()` for differential protein analysis.

**Input:** Loads `data_after_norm.pkl` (normalized data)  
**Output:** Saves `data_after_stat.pkl` + CSV results tables

In [ ]:
# Cell 1: Imports
import sys
sys.path.append('..')

from ipms.analysis import load_data, stat_ip
import pandas as pd

print("✓ All imports successful!")





# Cell 2: Load Normalized Data
data = load_data('../results/data_after_norm.pkl')

print(f"\n✓ Loaded {data['metadata']['n_proteins']} proteins")
print(f"Samples: {data['metadata']['n_samples']}")
print(f"Normalization: {data['normalization']['method']} + {data['normalization']['imputation']}")


# Cell 3: Run Statistical Analysis (Default Thresholds)
# p < 0.05, |log2FC| > 1.0

data = stat_ip(data, p_threshold=0.05, log2fc_threshold=1.0)

print("\n✓ Statistical analysis complete!")
print("\nResults saved with label: pval05_l2fc1")

In [ ]:
# Cell 4: Try More Stringent Thresholds (Optional)
# Uncomment to run with stricter criteria

# data = stat_ip(data, p_threshold=0.01, log2fc_threshold=2.0)
# print("Results saved with label: pval001_l2fc2")

print("(Uncomment above to try stringent thresholds)")

In [ ]:
# Cell 5: Try More Lenient Thresholds (Optional)
# Uncomment to run with looser criteria

# data = stat_ip(data, p_threshold=0.1, log2fc_threshold=0.5)
# print("Results saved with label: pval01_l2fc05")

print("(Uncomment above to try lenient thresholds)")

In [ ]:
# Cell 6: Inspect Statistical Results
results_df = data['stats_results']
sig_proteins = data['significant_proteins']
params = data['stats_params']

print("="*60)
print("STATISTICAL RESULTS")
print("="*60)

print(f"\nThresholds used:")
print(f"  P-value < {params['p_threshold']}")
print(f"  |Log2FC| > {params['log2fc_threshold']}")
print(f"  Correction: {params['correction']}")

print(f"\nSignificant proteins per comparison:")
for comparison, stats in sig_proteins.items():
    print(f"\n{comparison}:")
    print(f"  Total: {stats['total']}")
    print(f"  Enriched: {stats['enriched']}")
    print(f"  Depleted: {stats['depleted']}")

print(f"\nResults DataFrame shape: {results_df.shape}")
print(f"Columns: {list(results_df.columns)}")


# Cell 7: View Top Hits for All Comparisons
results_df = data['stats_results']

# Get all comparisons
comparison_cols = [c for c in results_df.columns if '_vs_' in c and '_log2FC' in c]
comparisons = [c.replace('_log2FC', '') for c in comparison_cols]

for comparison in comparisons:
    print(f"\nTop 10 enriched proteins in {comparison}:")
    print("="*60)
    
    # Filter for significant proteins
    sig_mask = results_df[f'{comparison}_significant'] == 'significant'
    
    # Sort by log2FC (descending) and show top 10
    top_enriched = results_df[sig_mask].nlargest(10, f'{comparison}_log2FC')
    
    display_cols = [
        'Gene_Symbol',
        f'{comparison}_log2FC',
        f'{comparison}_adj_pvalue',
        f'{comparison}_significant'
    ]
    
    if len(top_enriched) > 0:
        print(top_enriched[display_cols].to_string(index=False))
    else:
        print("No significant enriched proteins found")
    
    print()  # Blank line between comparisons




# Cell 8: Check Output Files
import os

tables_dir = '../results/tables'
threshold_label = data['stats_params']['threshold_label']

print("Statistical output files:")
print("="*60)

expected_files = [
    f'stats_results_{threshold_label}.csv',
    f'summary_{threshold_label}.csv'
]

# Add significant protein files
for comparison in data['significant_proteins'].keys():
    expected_files.append(f'{comparison}_significant_{threshold_label}.csv')

for filename in expected_files:
    filepath = os.path.join(tables_dir, filename)
    if os.path.exists(filepath):
        size_kb = os.path.getsize(filepath) / 1024
        # Count lines
        with open(filepath) as f:
            n_lines = sum(1 for _ in f) - 1  # -1 for header
        print(f"✓ {filename}")
        print(f"    {size_kb:.1f} KB, {n_lines} rows\n")
    else:
        print(f"✗ {filename} NOT FOUND\n")





# Cell 9: Compare Enriched Proteins Between Conditions
results_df = data['stats_results']
sig_proteins = data['significant_proteins']

if len(sig_proteins) > 1:
    print("Overlap between enriched proteins:")
    print("="*60)
    
    comparisons = list(sig_proteins.keys())
    
    for i, comp1 in enumerate(comparisons):
        for comp2 in comparisons[i+1:]:
            # Filter for ENRICHED only (positive log2FC)
            sig1_mask = (results_df[f'{comp1}_significant'] == 'significant') & (results_df[f'{comp1}_log2FC'] > 0)
            sig2_mask = (results_df[f'{comp2}_significant'] == 'significant') & (results_df[f'{comp2}_log2FC'] > 0)
            
            set1 = set(results_df[sig1_mask]['Accession'].tolist())
            set2 = set(results_df[sig2_mask]['Accession'].tolist())
            
            overlap = set1 & set2
            unique1 = set1 - set2
            unique2 = set2 - set1
            
            print(f"\n{comp1} vs {comp2} (enriched proteins only):")
            print(f"  Shared enriched: {len(overlap)}")
            print(f"  Unique to {comp1}: {len(unique1)}")
            print(f"  Unique to {comp2}: {len(unique2)}")
else:
    print("Only one comparison - no overlap analysis needed")





# Cell 10: Summary
print("="*60)
print("STATISTICAL ANALYSIS SUMMARY")
print("="*60)

print(f"\nProteins analyzed: {data['metadata']['n_proteins']}")
print(f"Comparisons performed: {len(data['significant_proteins'])}")

print(f"\nThresholds: {data['stats_params']['threshold_label']}")

print(f"\nData saved to:")
print(f"  • data_after_stat.pkl (for next step)")
print(f"  • stats_results_{data['stats_params']['threshold_label']}.csv")
print(f"  • [comparison]_significant_{data['stats_params']['threshold_label']}.csv")

print("\n" + "="*60)
print("NEXT STEP: 05_test_viz.ipynb for volcano plots and heatmaps")
print("="*60)

## Understanding the Results

### Columns in Results DataFrame:

For each comparison (e.g., `WT_vs_EV`):
- **`WT_vs_EV_log2FC`**: Log2 fold change (WT / EV)
  - Positive = enriched in WT
  - Negative = depleted in WT (enriched in EV)
- **`WT_vs_EV_pvalue`**: Raw p-value from t-test
- **`WT_vs_EV_adj_pvalue`**: FDR-corrected p-value (use this!)
- **`WT_vs_EV_control_mean`**: Mean intensity in control (EV)
- **`WT_vs_EV_treatment_mean`**: Mean intensity in treatment (WT)
- **`WT_vs_EV_significant`**: "significant" or "not_significant"

### Significance Criteria:

A protein is marked as **significant** if:
1. Adjusted p-value < threshold (default 0.05)
2. |Log2 fold change| > threshold (default 1.0)

Both criteria must be met!

### Output Files:

**`stats_results_pval05_l2fc1.csv`**
- All proteins with all statistics
- Use for custom filtering or further analysis

**`WT_vs_EV_significant_pval05_l2fc1.csv`**
- Only significant proteins for this comparison
- Ready for pathway analysis, literature search

**`summary_pval05_l2fc1.csv`**
- Quick overview of all comparisons
- Counts of enriched/depleted proteins

---

## Next Steps

After statistical analysis:
1. ✓ Review the significant protein lists
2. ✓ Check top enriched/depleted proteins
3. ✓ Proceed to **05_test_viz.ipynb** for volcano plots and heatmaps